In [ ]:
%pip install selenium

In [ ]:
%pip install pandas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Setup driver
path = r'C:\Users\ZA IT Park\Desktop\Hackaton\driver\chromedriver.exe'
service = Service(path)
driver = webdriver.Chrome(service=service)
driver.set_script_timeout(30)

driver.get('https://www.tradingview.com/markets/stocks-usa/market-movers-all-stocks/')

# Wait for the table rows to load before starting to scroll
wait = WebDriverWait(driver, 20) # wait untill condition not full fill
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'row-RdUXZpkv'))) #wait until not load in browser

# Step 2: Smooth scroll with pauses + Load More click
scroll_step = 500  # pixels
scroll_pause = 2   # seconds
max_scrolls = 100  # safety limit

scroll_count = 0
while scroll_count < max_scrolls:
    try:
        driver.execute_script(f"window.scrollBy(0, {scroll_step});") #scroll throw javascript
    except Exception as e:
        print(" Error during scrolling:", e)
        break

    time.sleep(scroll_pause)
    scroll_count += 1
    print(f"Scrolled {scroll_step * scroll_count} pixels")

    # Try clicking 'Load More' using class name
    try:
        load_more_button = driver.find_element(By.CLASS_NAME, "button-SFwfC2e0")
        if load_more_button.is_displayed():
            print("Clicking Load More button...")
            load_more_button.click()
            time.sleep(3)
    except:
        pass

    # Check if bottom of the page is reached
    try:
        new_scroll = driver.execute_script("return window.pageYOffset + window.innerHeight")  # how many time scroll by user & how many pixel scroll in check by view port height
        total_height = driver.execute_script("return document.body.scrollHeight") # check total scrollable height 
    except Exception as e:
        print(" Error executing JavaScript:", e)
        break

    if new_scroll >= total_height:
        print(" Reached bottom of the page.")
        break

# Step 3: Scrape data
findTr = driver.find_elements(By.CLASS_NAME, 'row-RdUXZpkv')

data = []

for dataSet in findTr[1:]: 
    td = dataSet.find_elements(By.TAG_NAME, 'td')
    if len(td) < 10:
        print("Skipping row with", len(td), "columns")
        continue

    try:
        symbol = td[0].find_element(By.TAG_NAME, 'a').text
        name = td[0].find_element(By.TAG_NAME, 'sup').text
    except:
        symbol = td[0].text
        name = ""

    price = td[1].text
    change_Price = td[2].text
    volume = td[3].text
    rel_volume = td[4].text
    market_Cap = td[5].text
    P_E = td[6].text 
    EPS_Growth_YoY = td[7].text
    Div_Yield = td[8].text
    Sector = td[9].text

    print(symbol, name, price, change_Price, volume, rel_volume, market_Cap, P_E)

    data.append({
        'Symbol': symbol,
        'Name': name,
        'Price': price,
        'Change Price': change_Price,
        'Volume': volume,
        'Relative Volume': rel_volume,
        'Market Cap': market_Cap,
        'P/E': P_E,
        'EPS Growth YoY': EPS_Growth_YoY,
        'Div Yield': Div_Yield,
        'Sector': Sector
    })

# Step 4: Save to Excel
df = pd.DataFrame(data)
df.to_excel("tradingview_data.xlsx", index=False)

driver.quit()
print("✅ Data saved to tradingview_data.xlsx")
